# IPEDS Completetions Data Download and Anaysis

<b>Install modules required</b>

In [ ]:
from pip._internal import main

<b>Defination to install modules if does not exists</b>

In [ ]:
def module_install(package):
    import importlib
    print('Checking for the package:', package)
    if (importlib.util.find_spec(package) is None):
        print('Installing package :******* ', package)
        try:
            main(['install', package])
        except Exception as exception:
            print('Unable to install : ', package)
            print('Exception message : ', exception.message)
    else:
        print('Already Installed : ', package)
    print('\n')

In [ ]:
module_install('pandas')
module_install('os')
module_install('sys')
module_install('requests')
module_install('zipfile')
module_install('io')

<b>Import required modules</b>

In [4]:
import pandas as pd
import os
import requests
import sys
import zipfile, io

<b>Declare global variable used to store Data and define Filename</b>

In [ ]:
file_name="Result.xlsx"
path=os.getcwd()+"\\data\\"
years=[]

<b>Method to take user input and create a folder for data download</b>

In [12]:
def iter_input():
    x, y = [int(x) for x in input("Enter the range of year you want to download the data:").split()] 
    
    try:
        os.mkdir(path)
    except OSError:
        print("Creation of the directory %s failed" % path)
    else:
        print("Successfully created the directory %s " % path)
    for i in range(x,y+1):
        print("Downloading file for the year :",i)
        download_file(i)

<b> Method to download the zip file of data and its dictionary for the year range specified by the user.</b>

In [1]:
def download_file(val):
    val=str(val)
    urls=["https://nces.ed.gov/ipeds/datacenter/data/C"+val+"_C.zip","https://nces.ed.gov/ipeds/datacenter/data/C"+val+"_C_Dict.zip"]
    for x in urls:
        url_response(x,val)

<b>Method to extract zip file if it exists</b>

In [17]:
def url_response(url,val):

    r = requests.get(url, stream=True)
    #print(r.status_code)
    if (r.status_code != 404):
        
        zip_ref = zipfile.ZipFile(io.BytesIO(r.content))
        zip_ref.extractall(path)
        zip_ref.close()
        years.append(val)
        #print(years)
    else:
        print("********File for this year not available!*******")

## Data Engineering

<b>Method for reading file and Calculating results for Number of students recieving degree/awards by gender and Race/Ethnicity</b>

In [15]:
def stat_add():
    #fetch
    years1 = list(set(years))
    
    print("Printing year that recorded:", years1)
    for i in range(len(years1)):
        first_data=pd.read_csv(path+"c"+years1[i]+"_c.csv")
        mapping = {1:'Award of less than 1 academic year',2:'Award of at least 1 but less than 4 academic years',3:'Associates degree',
                   5:'Bachelors degree',7:'Masters degree',9:'Doctors degree',10:'Postbaccalaureate or Post-masters certificate'}
        
        first_data.AWLEVELC=[mapping[item] for item in first_data.AWLEVELC]
        
        group = first_data.groupby(['AWLEVELC']).sum()

        group.rename(index=str, columns={"CSTOTLT": "TOTAL", "CSTOTLM": "TOTAL MEN", "CSTOTLW": "TOTAL WOMEN",
                                         "CSAIANT": "AMERICAN INDIAN OR ALASKA NATIVE", "CSASIAT": "ASIAN",
                                         "CSBKAAT": "BLACK OR AFRICAN AMERICAN",
                                         "CSHISPT": "HISPANIC OR LATIN",
                                         "CSNHPIT": "NATIVE HAWAIIN OR OTHER PACIFIC ISLANDER", "CSWHITT": "WHITE",
                                         "CSUNKNT": "UNKNOWN ETHNICITY", "CSNRALT": "NON ALEIN TOTAL"}, inplace=True)
        final = group[["TOTAL", "TOTAL MEN", "TOTAL WOMEN", "AMERICAN INDIAN OR ALASKA NATIVE", "ASIAN", "BLACK OR AFRICAN AMERICAN",
             "HISPANIC OR LATIN", "NATIVE HAWAIIN OR OTHER PACIFIC ISLANDER", "WHITE", "UNKNOWN ETHNICITY",
             "NON ALEIN TOTAL"]]
        #df.insert(-1, "Year", [val], True)
        
        final.insert(0,"Year",years1[i])
           
        if os.path.exists(file_name):
            final.to_csv(file_name, mode='a',header=False)
        else:
            final.to_csv(file_name, mode='a',header=True)

<b>Main Function</b>

In [19]:
if __name__ == "__main__":
    iter_input()
    stat_add()
    print("Program completed!")



Enter the range of year you want to download the data:2011 2017
Creation of the directory C:\Users\018807852SA\PycharmProjects\iped\venv\data\ failed
********File for this year not available!*******
********File for this year not available!*******
Printing year that recorded: ['2016', '2014', '2012', '2013', '2017', '2015']
Program completed!
